# First Project
Ollama -> Summary
huggingface_hub -> "facebook/m2m100_418M" for translation

In [ ]:
!pip install transformers datasets torch
!pip install huggingface_hub

In [ ]:
# imports
import requests
from bs4 import BeautifulSoup
import json
import ollama

In [ ]:
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

huggingface_url = "https://huggingface.co/learn/ml-for-3d-course"
huggingface_website = Website(huggingface_url)

huggingface_data = {
    "title": huggingface_website.title,
    "text": huggingface_website.text
}
print(huggingface_data)

with open('ml_for_3d_course_data.json', 'w') as f:
    json.dump(huggingface_data, f)


In [ ]:
# huggingface_data 'text' value
huggingface_text = huggingface_data['text']

# Summary
response_summary = ollama.chat(model="llama3.2:latest", messages=[{"role": "user", "content": f"Summarize the following text: {huggingface_text}"}])
print(response_summary)

# print summary
summary_huggingface_text = response_summary.message['content']
print("Summary Text:", summary_huggingface_text)



In [ ]:
# HuggingFace Translation

In [ ]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

url = "https://huggingface.co/learn/ml-for-3d-course"
website = Website(url)
print(website.title)  
print(website.text[:1000])

data = {
    "title": website.title,
    "text": website.text
}

with open('ml_for_3d_course_data.json', 'w') as f:
    json.dump(data, f)


In [ ]:
print(data)

In [ ]:
!pip install sacremoses

In [ ]:
import json
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load the M2M100 model and tokenizer
model_name = "facebook/m2m100_418M"
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)

# Load the saved JSON file
with open('ml_for_3d_course_data.json', 'r') as f:
    data = json.load(f)

# Extract text from the loaded data
text = data["text"]

# Set the source language to English and target language to Korean
source_lang = "en"
target_lang = "ko"

# Set the language for tokenizer (important for M2M100)
tokenizer.src_lang = source_lang
tokenizer.tgt_lang = target_lang

# Split text into smaller chunks if it's too large
# This step ensures we don't exceed the model's maximum length (512 tokens)
max_input_length = 512
chunks = [text[i:i+max_input_length] for i in range(0, len(text), max_input_length)]

print(chunks)
# Initialize a list to hold the translated text
translated_chunks = []

# Iterate through each chunk and translate it
for chunk in chunks:
    # Tokenize the chunk
    encoded = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Generate translation from the model, forcing the output to be in Korean
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id(target_lang), max_length=512)

    # Decode the translated tokens to text
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    translated_chunks.append(translated_text)

# Combine all translated chunks back together
final_translated_text = ' '.join(translated_chunks)
print("Translated Text:", final_translated_text)


In [ ]:
import json
import requests

# Ollama API URL 설정
ollama_url = "http://localhost:11411/v1/models/facebook/m2m100_418M/generate"

# 저장된 JSON 파일 로드
with open('ml_for_3d_course_data.json', 'r') as f:
    data = json.load(f)

# 텍스트 추출
course_text = data["text"]

# 번역할 소스 언어 및 타겟 언어 설정
source_language = "en"
target_language = "ko"

# 데이터 준비
payload = {
    "input_text": course_text,
    "src_lang": source_language,
    "tgt_lang": target_language
}

# API 호출
response = requests.post(ollama_url, json=payload)

# 응답 확인
if response.status_code == 200:
    translated_course_text = response.json().get("translated_text", "Translation failed")
    print("Translated Course Text:", translated_course_text)
else:
    print(f"Error {response.status_code}: {response.text}")
